In [20]:
from document_parsing.trec_car_parsing import TrecCarParser
from collections import namedtuple

parser = TrecCarParser()
path = '/Users/iain/LocalStorage/coding/github/entity-linking-with-pyspark/data/testY1_custom.bin'
doc_id = 'enwiki:Aftertaste'

Annotation = namedtuple('Annotation', ['anchor_text', 'anchor_text_location', 'entity_id', 'entity_name'])

AnchorTextLocation = namedtuple('AnchorTextLocation', ['start', 'end'])

document = parser.get_protobuf_message(path=path, doc_id=doc_id)

In [21]:
index_annotating = 15
possible_links = []


entity_ids = []
counter = 0
for document_content in document.document_contents:
    #for manual_entity_link in document_content.manual_entity_links:
    for manual_entity_link in document_content.synthetic_entity_links:
        entity_ids.append((manual_entity_link.entity_id, manual_entity_link.entity_name))
        
        counter += 1

valid_entity_ids = list(set(entity_ids))


def add_event_ent(matcher, doc, i, matches):
    # Get the current match and create tuple of entity label, start and end.
    # Append entity to the doc's entity. (Don't overwrite doc.ents!)
    match_id, start, end = matches[i]
    string_id = nlp.vocab.strings[match_id]
    span = doc[start:end] 
    print(match_id, string_id, start, end, span.text, span.start_char, span.end_char)
    

for i, document_content in enumerate(document.document_contents):
    for manual_entity_link in document_content.synthetic_entity_links:
        possible_links.append("{} | {}".format(manual_entity_link.entity_id, manual_entity_link.entity_name))

    if index_annotating == i:
        print('--- Metadata ---')
        print('mesage #: ' + str(i))
        print('content_id: ' + document_content.content_id)
        print('content_type: ' + str(document_content.content_type))

        print('\n------ Wikipedia Links ------')
        for manual_entity_link in document_content.synthetic_entity_links:
            
            print("{},\n".format(Annotation(
                anchor_text=manual_entity_link.anchor_text, 
                anchor_text_location=AnchorTextLocation(start=manual_entity_link.anchor_text_location.start, end=manual_entity_link.anchor_text_location.end), 
                entity_id=manual_entity_link.entity_id,
                entity_name=manual_entity_link.entity_name)))
        
        print('\n------ Text ------')
        text = document_content.text
        print(text)
        
#         print('\n------  Matcher ------')
#         doc = nlp(text=text)
#         matches = matcher(doc)
        print('\n======= Sentences =======')
        
        i_start = 0
        for i, sentence in enumerate(text.split('.')):
            
            print('\n------ Possible Links ------')
            for ps in sorted(valid_entity_ids):
                print(ps)
            
            print('== Sentences #{} =='.format(i))
            print(sentence)
            
            print('== Tokens =='.format(i))
            for token in sentence.split(' '): 
                i_end = i_start + len(token)
                print(token, i_start, i_end)
                i_start += len(token) + 1
            
        break
                                    

--- Metadata ---
mesage #: 15
content_id: a28ff3028b5669ed187a0a7138350af332ec7ed1
content_type: 1

------ Wikipedia Links ------
Annotation(anchor_text='tasting', anchor_text_location=AnchorTextLocation(start=8, end=15), entity_id='enwiki:Taste', entity_name='Taste'),

Annotation(anchor_text='tasting', anchor_text_location=AnchorTextLocation(start=98, end=105), entity_id='enwiki:Taste', entity_name='Taste'),

Annotation(anchor_text='taster', anchor_text_location=AnchorTextLocation(start=116, end=122), entity_id='enwiki:Taste', entity_name='Taste'),

Annotation(anchor_text='tasting', anchor_text_location=AnchorTextLocation(start=586, end=593), entity_id='enwiki:Taste', entity_name='Taste'),

Annotation(anchor_text='taster', anchor_text_location=AnchorTextLocation(start=116, end=122), entity_id='enwiki:Supertaster', entity_name='Supertaster'),

Annotation(anchor_text='flavor', anchor_text_location=AnchorTextLocation(start=665, end=671), entity_id='enwiki:Flavor', entity_name='Flavor'),


In [19]:
import re


In [4]:
text = \
"""  Aftertaste is the taste intensity of a food or beverage that is perceived immediately after that food or beverage
is removed from the mouth. The aftertastes of different foods and beverages can vary by intensity and over time,
but the unifying feature of aftertaste is that it is perceived after a food or beverage is either swallowed or
spat out. The neurobiological mechanisms of taste (and aftertaste) signal transduction from the taste receptors
in the mouth to the brain have not yet been fully understood. However, the primary taste processing area located
in the insula has been observed to be involved in aftertaste perception."""


In [109]:
for name in names:
new_name = name.lower()
new_text = text.lower()
pattern = r"{}".format(re.escape(new_name))
matches = [(m.start(0), m.end(0)) for m in re.finditer(pattern=pattern, string=new_text)]
for start, end in matches:
    print(text[start:end], start, end)

IndentationError: expected an indented block (<ipython-input-109-7cf43f870a89>, line 2)

In [125]:
# Lower
# Lemma
# multiple words

from nltk.stem import PorterStemmer

stemming = PorterStemmer()

def process_match_words(match_string):
    process_match_words = []
    match_words = match_string.split(" ")
    for w in match_words:
        w_stem = stemming.stem(w)
        if len(w_stem) > 0:
            process_match_words.append(w_stem)
    return process_match_words

match_string = "Octopus"

process_match_words = process_match_words(match_string=match_string)

match_string, process_match_words

('Octopus', ['octopu'])

In [131]:

start_i = 0
end_i = 0
matches = 0
required_matches = len(process_match_words)
for word_text in text.lower().split(" "):
    word_match = process_match_words[matches]
    if matches == 0:
        start_i_match = start_i
    if word_match in word_text: 
        matches += 1
    else:
        matches = 0
    
    # next word
    end_i += len(word_text) 
    assert word_text == text.lower()[start_i:end_i], "word_text: {}, text[start_i:end_i]: {}".format(word_text, text.lower()[start_i:end_i])

    
    if matches == required_matches:
        print("MATCH")
        print(text.lower()[start_i_match:end_i])
        matches = 0
    # space
    end_i += 1
    start_i = end_i
    

    

MATCH
aftertaste
MATCH
aftertastes
MATCH
aftertaste
MATCH
aftertaste)
MATCH
aftertaste


In [117]:
"   Aftertaste  i".split(" ")

['', '', '', 'Aftertaste', '', 'i']

In [10]:

['a', 'db'] in [['a', 'b'],['a', 'c']]

False

In [5]:
c = {'a': [1,2]}

In [6]:
c['a'].append(3)

In [7]:
c

{'a': [1, 2, 3]}

[['a', 'b'], ['a'], ['b', 'a', 'a'], ['c', 'b'], ['d', 'e']]

In [97]:
def get_unique_inputs(test_list):
    
    test_list_new = test_list
    for seq_a in test_list:
    
        seq_a_str = ''.join(seq_a)
        
        for seq_b in test_list:
            seq_b_str = ''.join(seq_b)
            
            if seq_a_str != seq_b_str:
            
                if (seq_a_str in seq_b_str) or (seq_b_str in seq_a_str):
                    if (len(seq_a_str) - len(seq_b_str)) > 0:
                        if seq_a in test_list_new:
                            test_list_new.remove(seq_a)
                    else:
                        if seq_b in test_list_new:
                            test_list_new.remove(seq_b)
    return test_list_new
        

test_list = [['a','b'], ['a'], ['b','b','a'], ['c','b'], ['d','e'], ['b', 'b']]
test_list_result = [['a'], ['c','b'], ['d','e'], ['b', 'b']]
assert get_unique_inputs(test_list=test_list) == test_list_result

In [98]:
test_list = [['a','b'], ['a'], ['b','b','a'], ['c','b'], ['d','e'], ['b', 'b']]
test_list_result = [['a'], ['c','b'], ['d','e'], ['b', 'b']]
get_unique_inputs(test_list=test_list)

[['a'], ['c', 'b'], ['d', 'e'], ['b', 'b']]

In [ ]:
['a', 'b']

In [66]:
'a' in 'a'

True

In [77]:
min(['1', '1'], ['2'])

['1', '1']

In [90]:
test_list.remove(['a'])

In [91]:
test_list


[['a', 'b'], ['b', 'b', 'a'], ['c', 'b'], ['d', 'e'], ['b', 'b']]

In [104]:
'abcde'[:3]

'abc'

In [16]:
text = 'sdvanoierbolefinb qn.fe iadfban. uieig sng '
start_i_match = 1
end_i = 21
while text[end_i-1].isalpha() == False:
    end_i += -1
    print(text[start_i_match:end_i])


dvanoierbolefinb qn


In [17]:


'é'.isalpha()


True

In [20]:
"Marsyas under Apollo's punishment, İstanbul Archaeology Museum".lower().split(" ")

['marsyas',
 'under',
 "apollo's",
 'punishment,',
 'i̇stanbul',
 'archaeology',
 'museum']

In [22]:
text = 'arsyas'
text[:len(text)]

'arsyas'

In [33]:
text = 'İstanbul'
print(text)
print(len(text))

İstanbul
8


In [35]:
text_lower = text.lower()
print(text_lower)
print(len(text_lower))

i̇stanbul
9


In [37]:
"asd sdv dfv"[5]

'd'

In [54]:
from nltk.stem import LancasterStemmer
stemmer = LancasterStemmer()
stemmer.stem('horned')

'horn'

'horni'

In [51]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=False)
stemmer.stem('horny,')

ImportError: cannot import name 'LancasterStemmer' from 'nltk.stem.snowball' (/Users/iain/LocalStorage/virtualenvs/entity_linking_env/lib/python3.7/site-packages/nltk/stem/snowball.py)

In [57]:
from string import punctuation
s = '.$loves,#'

print(s.strip(punctuation))

loves


In [58]:
text = 'Popular culture'

In [59]:
import urllib

In [63]:
urllib.parse.quote(string=, encoding='utf-8')

'Popular%20culture'

In [70]:
'asd  vsdvafdvbdfbsg fsgiuhg984w3RGEOadfb, '.rstrip()

'asd  vsdvafdvbdfbsg fsgiuhg984w3RGEOadfb,'